In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

## Loading the document and spliting them

In [2]:
loader=TextLoader('speech.txt')
doc=loader.load()


In [3]:
doc

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [8]:
text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)

In [9]:
docs=text_splitter.split_documents(doc)

In [10]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

## Creating the Embedding and VectorDB

In [12]:
embeddings=(
    OllamaEmbeddings(model="gemma:2b") ## by default it uses llama2
)
db=FAISS.from_documents(docs, embeddings)
db

In [13]:
### Quering 
quary="What does the speaker believe is the main reason the united states should enter the war?"

In [14]:
docs_result=db.similarity_search(quary)
docs_result

[Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her

In [16]:
docs_result[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

## As a Retriever
- We can also convert the vectorstore into a Retriever class. This allow us to easily use it in other LangChain methods, which largely work with retrievers

In [18]:
reteriver=db.as_retriever()
reterv_result=reteriver.invoke(quary)

In [20]:
reterv_result[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

## Similarity Search with Score
- There are some FAISS specific methods. One of them is similarity search with score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better

In [21]:
db.similarity_search_with_score(quary)

[(Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  3332.566),
 (Document(metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileg

### Send vector instead Vector

In [22]:
embedding_vector=embeddings.embed_query(quary)

In [23]:
embedding_vector

[-0.09209811687469482,
 1.3883330821990967,
 0.41739028692245483,
 1.025728464126587,
 0.36897915601730347,
 0.7780635952949524,
 0.6148719191551208,
 0.09608729928731918,
 0.18176710605621338,
 0.09652157127857208,
 -1.217602014541626,
 -0.17150098085403442,
 -0.48032456636428833,
 1.2569199800491333,
 1.329684853553772,
 -0.4118184745311737,
 2.2988858222961426,
 1.3665971755981445,
 1.5001707077026367,
 0.7953564524650574,
 0.7519111037254333,
 -0.5530590415000916,
 1.5334688425064087,
 0.5998924970626831,
 -0.5757286548614502,
 -0.8474978804588318,
 -1.5327407121658325,
 -1.70359206199646,
 -1.1777111291885376,
 -1.896048903465271,
 2.701615810394287,
 -0.6126877665519714,
 1.35452401638031,
 -0.4391549825668335,
 -0.7808471918106079,
 -0.4416394829750061,
 0.8947708010673523,
 -0.79776930809021,
 0.036135487258434296,
 -0.5185542702674866,
 -0.5694722533226013,
 -0.3622376322746277,
 0.5826745629310608,
 -0.40288519859313965,
 -1.2708661556243896,
 0.9163869619369507,
 0.369336277

In [24]:
docs_score=db.similarity_search_by_vector(embedding_vector)

In [25]:
docs_score

[Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(metadata={'source': 'speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when America is privileged to spend her

### Saving and loading vectorDB

In [26]:
db.save_local("faiss_index")

In [28]:
new_db=FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
new_db